In [ ]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
# Load the TensorBoard notebook extension
%load_ext tensorboard

SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [195]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [ ]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [ ]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [ ]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [ ]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

26


In [ ]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(4, 1): (shimmered, the)
(2, 1): (wide, the)
(4, 5): (shimmered, in)
(2, 4): (wide, shimmered)
(5, 4): (in, shimmered)


In [ ]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([1 7 5 0], shape=(4,), dtype=int64)
['the', 'sun', 'in', '<pad>']


In [ ]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word

In [ ]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 4
target_word     : shimmered
context_indices : [1 1 7 5 0]
context_words   : ['the', 'the', 'sun', 'in', '<pad>']
label           : [1 0 0 0 0]


In [ ]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : 4
context : tf.Tensor([1 1 7 5 0], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [ ]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

Извлечение текста из документа

In [ ]:
from spire.doc import *
from spire.doc.common import *

# Create a Document object
document = Document()
# Load a Word document
document.LoadFromFile("../data/Instructions1CEcology-Belov.docx")

path_to_file = "../data/Output/DocumentText.txt"
# Extract the text of the document
document_text = document.GetText()

# Write the extracted text into a text file
with open(path_to_file, "w", encoding="utf-8") as file:
    file.write(document_text)

document.Close()

In [151]:
from tika import parser  # pip install tika


raw = parser.from_file('../data/instrukciya-po-ehkspluatacii-v8.pdf')
text = raw['content']



# Разделить текст на строки
lines = text.splitlines()

lines = [line for line in lines if len(line.strip()) > 10]

text = '\n'.join(lines)



with open("../data/Output/DocumentTextPdf.txt", "w", encoding="utf-8") as f:
    f.write(text)


lines = text.splitlines()
for line in lines[:20]:
    print(line)


1С:Предприятие 8.3. Руководство пользователя
по эксплуатации 
"1С:ПРЕДПРИЯТИЕ 8" 
ПРАВО ТИРАЖИРОВАНИЯ  
ПРОГРАММНЫХ СРЕДСТВ И ДОКУМЕНТАЦИИ 
ПРИНАДЛЕЖИТ ФИРМЕ «1С» 
Приобретая систему «1С:Предприятие»,  
вы тем самым даете согласие не допускать 
копирования программ и документации 
без письменного разрешения фирмы «1С» 
© ООО «1С», 2020 
Фирма «1С», Москва, 123056, а/я 64. 
Отдел продаж: Селезневская ул., 21, 
телефон: (495) 737-92-57, 
факс: (495) 681-44-07. 
E-mail: 1c@1c.ru 
URL: www.1c.ru, www.v8.1c.ru 
Группа разработки программ – А. Алексеев, А. Безбородов, Д. Бескоровайнов, 
П. Василец, А. Виноградов, А. Волков, И. Гольштейн, Е. Горностаев, 
Г. Дамье, А. Даровских, О. Дерут, Н. Евграфов, Б. Евтифеев, Д. Зарецкий, 


In [152]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)


Evaluation Warning: The document was created with Spire.Doc for Python.
Общая информация
Основным назначением информационной системы 1С Экология является автоматизация задач экологической безопасности и охраны окружающей среды в крупных территориально-распределенных и холдинговых компаниях. 

Бизнес-процессы, реализованные в функционале 1С Экология:
* Учет – ведение учета по всем направлениям в области охраны окружающей среды;
* Планирование – составление годовых программ и планов;
* Контроль – подготовка протоколов и актов о проведенных мероприятиях по охране окружающей среды;
* Формирование отчетности – формирование основных документов для отчета перед госорганами.
Инструкция предназначена для сотрудников, работающих в системе 1С Экология для ведения отчетности по всем направлениям защиты окружающей среды.
Термины и определения
ООС – охрана окружающей среды
ИС – информационная система
ПСО – Портал самообслуживания, https://pso.enplus.group/
ЗВ – загрязняющее вещество
ИЗА – источники 

In [153]:
# Откройте первый файл и прочтите его содержимое
with open(path_to_file, 'r', encoding='utf-8') as file1:
    content1 = file1.read()

# Откройте второй файл и прочтите его содержимое
with open('../data/Output/DocumentTextPdf.txt', 'r', encoding='utf-8') as file2:
    content2 = file2.read()

# Объедините содержимое двух файлов
merged_content = content1 + '\n' + content2

# Запишите объединенное содержимое в новый файл
with open(path_to_file, 'w', encoding='utf-8') as merged_file:
    merged_file.write(merged_content)


In [154]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [155]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [156]:
vectorize_layer.adapt(text_ds.batch(1024))

2024-05-13 06:43:30.529958: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [157]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'в', 'и', '–', 'для', 'с', 'на', '\uf0b7', 'или', 'по', 'Для', 'В', 'можно', 'при', 'не', 'Если', 'то', 'документа', 'будет']


In [158]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [159]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

10311


2024-05-13 06:43:32.074460: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [160]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[   1    1 3283    1    1    1    1    1 3300    1] => ['[UNK]', '[UNK]', 'the', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'for', '[UNK]']
[  1 188   0   0   0   0   0   0   0   0] => ['[UNK]', 'информация', '', '', '', '', '', '', '', '']
[   1    1  126   92 1852 2501  358    1  894    1] => ['[UNK]', '[UNK]', 'информационной', 'системы', '1С', 'Экология', 'является', '[UNK]', 'задач', '[UNK]']
[ 839    1    2    1 1852 2501    0    0    0    0] => ['Бизнеспроцессы', '[UNK]', 'в', '[UNK]', '1С', 'Экология', '', '', '', '']
[ 679    4 1766  334   10 1049    1    2   52 3972] => ['Учет', '–', 'ведение', 'учета', 'по', 'всем', '[UNK]', 'в', 'области', 'охраны']


In [161]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")



  4%|▍         | 429/10311 [00:00<00:04, 2153.48it/s]

100%|██████████| 10311/10311 [00:07<00:00, 1468.99it/s]




targets.shape: (29540,)
contexts.shape: (29540, 5)
labels.shape: (29540, 5)


In [162]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)


<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [163]:
# dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
# print(dataset)



In [164]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [165]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [166]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [167]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [168]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2268 - loss: 1.6088
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6577 - loss: 1.5949
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8259 - loss: 1.5774
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8684 - loss: 1.5466
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8735 - loss: 1.4926
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8680 - loss: 1.4145
Epoch 7/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8681 - loss: 1.3227
Epoch 8/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8699 - loss: 1.2267
Epoch 9/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8803 - loss: 1.1255
Epoch 10/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8855 - loss: 1.0320
Epoch 11/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8924 - loss: 0.9425
Epoch 12/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9057 - loss: 0.8519


In [169]:
#docs_infra: no_execute
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 25706), started 0:08:17 ago. (Use '!kill 25706' to kill it.)

In [170]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [171]:
out_v = io.open('vectors.csv', 'w', encoding='utf-8')
out_m = io.open('metadata.csv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [172]:
pip install langdetect


Note: you may need to restart the kernel to use updated packages.


In [173]:
# import pandas as pd

# # Открываем файл для записи
# with open('metadataforfind.csv', 'w', encoding='utf-8') as out_m:
#     # Записываем заголовки столбцов
#     out_m.write("ID,Text\n")

#     # Записываем данные в файл
#     for index, word in enumerate(vocab):
#         out_m.write(f"{index},{word}\n")

# # Создаем DataFrame из записанных данных
# df = pd.read_csv('metadata.csv')

# # Записываем DataFrame в файл CSV без индексов
# df.to_csv('metadataforfind.csv', index=False)


In [174]:
out_m = io.open('metadataforfind.csv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_m.write(str(index)+word + "\n")
out_m.close()


In [175]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

In [176]:
pip install gensim nltk


Note: you may need to restart the kernel to use updated packages.


In [177]:
pip install scipy==1.12


Note: you may need to restart the kernel to use updated packages.


In [178]:
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
def sent_alter(model, sentence=""):
  wordsIn = word_tokenize(sentence.lower())
  wordsOut=[]
  for w in wordsIn:
    try:
      wordsOut.append(most_sim(model,w,1)[0][0])
    except:
      wordsOut.append(w)
  resultSentence = " ".join(wordsOut)
  return resultSentence

In [179]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [180]:
sentence = "Неоьходима замена интерфейса"
altered_sentence = sent_alter(word2vec, sentence)
print("Original Sentence:", sentence)
print("Altered Sentence:", altered_sentence)

Original Sentence: Неоьходима замена интерфейса
Altered Sentence: неоьходима замена интерфейса


In [181]:
pip install langchain[all] Langchain[FAISS] faiss-cpu # WARNING: langchain 0.0.74 does not provide the extra 'faiss'

Note: you may need to restart the kernel to use updated packages.


In [182]:
from langchain import FAISS
import pandas as pd
import numpy as np
import os
import time
import pickle

In [183]:
import csv
with open('./metadata.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

[UNK]
в
и
–
для
с
на

или
по
Для
В
можно
при
не
Если
то
документа
будет
При
выбрать
из
следует
нажать
списка
от
к
нужно
настройки
кнопку
табличного
если
может
окна
панели
меню
строки
текста
данных
элементов
помощью
пункт
отчета
элемента
быть
как
Чтобы
группы
значение
поле
поля
области
работы
этом
а
мыши
формы
свойств
значения
строк
объектов
группировки
Настройка
будут
кнопки
выбора
колонок
все
действия
только
необходимо
колонки
ввода
записи
списке
окно
что
также
список
о
использовать
ячейки
текст
установить
схемы
тип
этого
документов
документ
окне
На
системы
ячеек
которые
Таблица
настроек
команды
используется
объекта
форме
отбора
могут
инструментов
режим
данные
печати
за
его
«1СПредприятие»
таблицы
позволяет
значений
Работа
конфигурации
их
чтобы
режиме
параметров
нажатии
которой
клавиши
Например
экран
картинки
1
информационной
диалоге
выделить
элемент
Все

просмотра
производится
которых
качестве
элементы
полей
ее
указать
строку
страницы
команд
редактирования
x
окон
объект
например
им

In [184]:
passage_data = pd.read_csv("./metadata.csv")

In [185]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [186]:
from langchain.embeddings import SentenceTransformerEmbeddings 
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/home/codespace/.python/current/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [187]:
import time
start=time.time() 
metadatas = [] 
for index, row in passage_data.iterrows(): 
  doc_meta = {  } 
  metadatas.append(doc_meta)

In [188]:
pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Note: you may need to restart the kernel to use updated packages.


In [206]:
def tokenize(text):
    vocab = {}
    index = 0
    
    # Разделяем текст на слова
    tokens = text.split()
    
    # Добавляем паддинг токен
    vocab['<pad>'] = index
    index += 1
    
    # Индексируем остальные слова
    for token in tokens:
        if token.lower() not in vocab:
            vocab[token.lower()] = index
            index += 1
    
    # Размер словаря
    vocab_size = len(vocab)
    
    return vocab

In [235]:
import csv

def load_csv_to_dict(file_path, key_column):
    result = {}
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            key = row[key_column]
            result[key] = row
    return result

# Example usage:
csv_file_path = './metadata.csv'
key_column_index = 0  # Assuming the first column is the key column
vocab_dict = load_csv_to_dict(csv_file_path, key_column_index)
print(vocab_dict)


{'в': ['в'], 'и': ['и'], '–': ['–'], 'для': ['для'], 'с': ['с'], 'на': ['на'], '\uf0b7': ['\uf0b7'], 'или': ['или'], 'по': ['по'], 'Для': ['Для'], 'В': ['В'], 'можно': ['можно'], 'при': ['при'], 'не': ['не'], 'Если': ['Если'], 'то': ['то'], 'документа': ['документа'], 'будет': ['будет'], 'При': ['При'], 'выбрать': ['выбрать'], 'из': ['из'], 'следует': ['следует'], 'нажать': ['нажать'], 'списка': ['списка'], 'от': ['от'], 'к': ['к'], 'нужно': ['нужно'], 'настройки': ['настройки'], 'кнопку': ['кнопку'], 'табличного': ['табличного'], 'если': ['если'], 'может': ['может'], 'окна': ['окна'], 'панели': ['панели'], 'меню': ['меню'], 'строки': ['строки'], 'текста': ['текста'], 'данных': ['данных'], 'элементов': ['элементов'], 'помощью': ['помощью'], 'пункт': ['пункт'], 'отчета': ['отчета'], 'элемента': ['элемента'], 'быть': ['быть'], 'как': ['как'], 'Чтобы': ['Чтобы'], 'группы': ['группы'], 'значение': ['значение'], 'поле': ['поле'], 'поля': ['поля'], 'области': ['области'], 'работы': ['работы'

In [238]:
import faiss
import time

weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

def langchain_vector_search(index_path, query_text,vocab, k=5):
    # Загрузка индекса Langchain
    loaded_index = faiss.read_index(index_path)

    # Преобразование текста запроса в вектор с помощью Langchain
    query_vector = tokenize(query_text,vocab)

    # Выполнение поиска похожих векторов
    D, I = loaded_index.search(query_vector, k)

    # Возвращаем индексы и оценки (scores) найденных векторов
    return D, I


def tokenize(text, vocab):
    tokens = text.split()
    token_indices = []
    for token in tokens:
        if token.lower() in vocab:
            token_indices.append(vocab[token.lower()])
        else:
            token_indices.append(0)  # Индекс паддинга для неизвестных слов
    return token_indices




# Пример использования функции
index_path = "./LangChain_FAISS"
query_text = "What is atheism?"
k = 5
start_time = time.time()
distances, indices = langchain_vector_search(index_path, query_text,vocab_dict, k)
end_time = time.time()

# Вывод результатов
print(f"Top {k} similar vectors to the query '{query_text}':")
for i, (distance, index) in enumerate(zip(distances[0], indices[0])):
    print(f"Rank {i+1}: Vector index {index}, Distance {distance}")

print(f"Search execution time: {end_time - start_time} seconds")


AttributeError: 'list' object has no attribute 'shape'

In [196]:
start=time.time() 
loaded_index.similarity_search_with_score("What is atheism?",5)

AttributeError: 'IndexFlatL2' object has no attribute 'similarity_search_with_score'

In [197]:
print(list("What is atheism?".lower().split()))

['what', 'is', 'atheism?']
